In [1]:
# Load all packages necessary for analysis
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import timeit
import filecmp
import url_stripper

# deck_urls

## Gather

In [2]:
# get a list of file names in type_html_files folder and name it dir_names
dir_names = []
cur_dir = 'C:/Users/muroc/Documents/MTG/type_html_files/'

with os.scandir(cur_dir) as folder:
    for file in folder: 
        if file.is_file():
            dir_names.append(file.name)

In [15]:
# get a list of all urls in all files
deck_urls = pd.DataFrame(columns=['player', 'url'])

for i in np.arange(np.count_nonzero(dir_names)):
    with open('C:/Users/muroc/Documents/MTG/type_html_files/' + dir_names[i]) as file:
        soup = BeautifulSoup(file, 'lxml')
    data = pd.DataFrame({'player': soup.find_all(href=re.compile(r'search\?player')), 
                         'url': soup.find_all(href=re.compile(r'event\?e\=.+\&d\=.+\&f\=ST'))})
    deck_urls = deck_urls.append(data)

,player,url


In [ ]:
# turn deck urls into a dataframe
deck_urls = {'url': deck_urls}
deck_urls = pd.DataFrame(data=deck_urls)

In [ ]:
# get deck names and clean them up
url_stripper.url_stripper(deck_urls, 'url', 'name', '(>.+<)')

In [ ]:
# extract url from the url column
deck_urls['url'] = deck_urls.url.str.extract('(event.+f\=ST)')

In [ ]:
# add root url to the data
urls = []
names = []

for i in np.arange(deck_urls.shape[0]):
    urls.append('https://www.mtgtop8.com/' + str(deck_urls.url[i]))
    names.append(str(deck_urls.name[i]) + '.txt')

In [ ]:
# clean urls and name
deck_urls['url'] = urls
deck_urls['name'] = names
deck_urls['name'] = deck_urls.name.str.replace(' ', '_')
deck_urls['name'] = deck_urls.name.str.replace('/', '-')

In [ ]:
deck_urls['player'] = player_names

In [ ]:
url_stripper.url_stripper(deck_urls, 'player', 'player', '(>.+<)')

In [ ]:
deck_urls.player = deck_urls.player.str.replace(' ', '_')

In [ ]:
deck_urls.to_csv('C:/Users/muroc/Documents/MTG/data/deck_urls.csv', index=False)